# Data from https://github.com/CSSEGISandData/COVID-19

In [1]:
import pandas as pd
from sqlalchemy import create_engine

In [2]:
#data
confirmed = "assets/data/time_series_19-covid-Confirmed.csv"
deaths = "assets/data/time_series_19-covid-Deaths.csv"
recovered = "assets/data/time_series_19-covid-Recovered.csv"


In [3]:
# create data frames
confirmed_df = pd.read_csv(confirmed)
deaths_df = pd.read_csv(deaths)
recovered_df = pd.read_csv(recovered)
confirmed_df.head(10)
# tot_locations = len(confirmed_df)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,3/6/20,3/7/20,3/8/20,3/9/20,3/10/20,3/11/20,3/12/20,3/13/20,3/14/20,3/15/20
0,NaN,Thailand,15.0000,101.0000,2,3,5,7,8,8,...,48,50,50,50,53,59,70,75,82,114
1,NaN,Japan,36.0000,138.0000,2,1,2,2,4,4,...,420,461,502,511,581,639,639,701,773,839
2,NaN,Singapore,1.2833,103.8333,0,1,3,3,4,5,...,130,138,150,150,160,178,178,200,212,226
3,NaN,Nepal,28.1667,84.2500,0,0,0,1,1,1,...,1,1,1,1,1,1,1,1,1,1
4,NaN,Malaysia,2.5000,112.5000,0,0,0,3,4,4,...,83,93,99,117,129,149,149,197,238,428
5,British Columbia,Canada,49.2827,-123.1207,0,0,0,0,0,0,...,21,21,27,32,32,39,46,64,64,73
6,New South Wales,Australia,-33.8688,151.2093,0,0,0,0,3,4,...,26,28,38,48,55,65,65,92,112,134
7,Victoria,Australia,-37.8136,144.9631,0,0,0,0,1,1,...,10,11,11,15,18,21,21,36,49,57
8,Queensland,Australia,-28.0167,153.4000,0,0,0,0,0,0,...,13,13,15,15,18,20,20,35,46,61
9,NaN,Cambodia,11.5500,104.9167,0,0,0,0,0,1,...,1,1,2,2,2,3,3,5,7,7


In [4]:
# Reset Index to have ID column
confirmed_df.reset_index(inplace=True)
confirmed_df.head()

,index,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,...,3/6/20,3/7/20,3/8/20,3/9/20,3/10/20,3/11/20,3/12/20,3/13/20,3/14/20,3/15/20
0,0,NaN,Thailand,15.0000,101.0000,2,3,5,7,8,...,48,50,50,50,53,59,70,75,82,114
1,1,NaN,Japan,36.0000,138.0000,2,1,2,2,4,...,420,461,502,511,581,639,639,701,773,839
2,2,NaN,Singapore,1.2833,103.8333,0,1,3,3,4,...,130,138,150,150,160,178,178,200,212,226
3,3,NaN,Nepal,28.1667,84.2500,0,0,0,1,1,...,1,1,1,1,1,1,1,1,1,1
4,4,NaN,Malaysia,2.5000,112.5000,0,0,0,3,4,...,83,93,99,117,129,149,149,197,238,428


In [5]:
# Rename to ID column
confirmed_df = confirmed_df.rename(columns={"index":"id"})
confirmed_df.head()

,id,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,...,3/6/20,3/7/20,3/8/20,3/9/20,3/10/20,3/11/20,3/12/20,3/13/20,3/14/20,3/15/20
0,0,NaN,Thailand,15.0000,101.0000,2,3,5,7,8,...,48,50,50,50,53,59,70,75,82,114
1,1,NaN,Japan,36.0000,138.0000,2,1,2,2,4,...,420,461,502,511,581,639,639,701,773,839
2,2,NaN,Singapore,1.2833,103.8333,0,1,3,3,4,...,130,138,150,150,160,178,178,200,212,226
3,3,NaN,Nepal,28.1667,84.2500,0,0,0,1,1,...,1,1,1,1,1,1,1,1,1,1
4,4,NaN,Malaysia,2.5000,112.5000,0,0,0,3,4,...,83,93,99,117,129,149,149,197,238,428


In [6]:
#Transpose Columns to get Dates as Row Values
df_melt = confirmed_df.melt(id_vars=['id','Province/State','Country/Region','Lat','Long'])
df_melt.columns = ['id','Province/State','Country/Region','Lat','Long','Date','confirmed_to_date']
df_melt['Date'] = pd.to_datetime(df_melt['Date'])
df_melt = df_melt.sort_values(['Date','id'])
df_melt['Count_Prev_Day'] = df_melt.sort_values('Date').groupby(['id','Province/State','Country/Region',
                                                                 'Lat','Long'])['confirmed_to_date'].shift(1)
df_melt[['Count_Prev_Day']] = df_melt[['Count_Prev_Day']].fillna(value=0)
df_melt['new_confirmed'] = df_melt['confirmed_to_date'] - df_melt['Count_Prev_Day']

df_melt.head()

,id,Province/State,Country/Region,Lat,Long,Date,confirmed_to_date,Count_Prev_Day,new_confirmed
0,0,NaN,Thailand,15.0000,101.0000,2020-01-22,2,0.0,2.0
1,1,NaN,Japan,36.0000,138.0000,2020-01-22,2,0.0,2.0
2,2,NaN,Singapore,1.2833,103.8333,2020-01-22,0,0.0,0.0
3,3,NaN,Nepal,28.1667,84.2500,2020-01-22,0,0.0,0.0
4,4,NaN,Malaysia,2.5000,112.5000,2020-01-22,0,0.0,0.0


In [7]:
#Check one location to make sure daily count working correctly
df_melt[df_melt['Province/State'] == 'Beijing'].sort_values('Date')

,id,Province/State,Country/Region,Lat,Long,Date,confirmed_to_date,Count_Prev_Day,new_confirmed
172,172,Beijing,China,40.1824,116.4142,2020-01-22,14,0.0,14.0
622,172,Beijing,China,40.1824,116.4142,2020-01-23,22,14.0,8.0
1072,172,Beijing,China,40.1824,116.4142,2020-01-24,36,22.0,14.0
1522,172,Beijing,China,40.1824,116.4142,2020-01-25,41,36.0,5.0
1972,172,Beijing,China,40.1824,116.4142,2020-01-26,68,41.0,27.0
2422,172,Beijing,China,40.1824,116.4142,2020-01-27,80,68.0,12.0
2872,172,Beijing,China,40.1824,116.4142,2020-01-28,91,80.0,11.0
3322,172,Beijing,China,40.1824,116.4142,2020-01-29,111,91.0,20.0
3772,172,Beijing,China,40.1824,116.4142,2020-01-30,114,111.0,3.0
4222,172,Beijing,China,40.1824,116.4142,2020-01-31,139,114.0,25.0


In [8]:
#Drop Count Prev Day column
confirmed_df = df_melt.drop(columns=['Count_Prev_Day'])
confirmed_df.head()

,id,Province/State,Country/Region,Lat,Long,Date,confirmed_to_date,new_confirmed
0,0,NaN,Thailand,15.0000,101.0000,2020-01-22,2,2.0
1,1,NaN,Japan,36.0000,138.0000,2020-01-22,2,2.0
2,2,NaN,Singapore,1.2833,103.8333,2020-01-22,0,0.0
3,3,NaN,Nepal,28.1667,84.2500,2020-01-22,0,0.0
4,4,NaN,Malaysia,2.5000,112.5000,2020-01-22,0,0.0


In [9]:
#Reset index to have ID column
recovered_df.reset_index(inplace=True)
recovered_df.head()

,index,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,...,3/6/20,3/7/20,3/8/20,3/9/20,3/10/20,3/11/20,3/12/20,3/13/20,3/14/20,3/15/20
0,0,NaN,Thailand,15.0000,101.0000,0,0,0,0,2,...,31,31,31,31,33,34,34,35,35,35
1,1,NaN,Japan,36.0000,138.0000,0,0,0,0,1,...,46,76,76,76,101,118,118,118,118,118
2,2,NaN,Singapore,1.2833,103.8333,0,0,0,0,0,...,78,78,78,78,78,96,96,97,105,105
3,3,NaN,Nepal,28.1667,84.2500,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
4,4,NaN,Malaysia,2.5000,112.5000,0,0,0,0,0,...,22,23,24,24,24,26,26,26,35,42


In [10]:
#Rename to have id column
recovered_df = recovered_df.rename(columns={"index":"id"})
recovered_df.head()

,id,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,...,3/6/20,3/7/20,3/8/20,3/9/20,3/10/20,3/11/20,3/12/20,3/13/20,3/14/20,3/15/20
0,0,NaN,Thailand,15.0000,101.0000,0,0,0,0,2,...,31,31,31,31,33,34,34,35,35,35
1,1,NaN,Japan,36.0000,138.0000,0,0,0,0,1,...,46,76,76,76,101,118,118,118,118,118
2,2,NaN,Singapore,1.2833,103.8333,0,0,0,0,0,...,78,78,78,78,78,96,96,97,105,105
3,3,NaN,Nepal,28.1667,84.2500,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
4,4,NaN,Malaysia,2.5000,112.5000,0,0,0,0,0,...,22,23,24,24,24,26,26,26,35,42


In [11]:
#transpose date columns to row values
df_melt = recovered_df.melt(id_vars=['id','Province/State','Country/Region','Lat','Long'])
df_melt.columns = ['id','Province/State','Country/Region','Lat','Long','Date','recovered_to_date']
df_melt['Date'] = pd.to_datetime(df_melt['Date'])
df_melt = df_melt.sort_values(['Date','id'])
df_melt['Count_Prev_Day'] = df_melt.sort_values('Date').groupby(['id','Province/State','Country/Region',
                                                                 'Lat','Long'])['recovered_to_date'].shift(1)
df_melt[['Count_Prev_Day']] = df_melt[['Count_Prev_Day']].fillna(value=0)
df_melt['new_recovered'] = df_melt['recovered_to_date'] - df_melt['Count_Prev_Day']

df_melt.head()

,id,Province/State,Country/Region,Lat,Long,Date,recovered_to_date,Count_Prev_Day,new_recovered
0,0,NaN,Thailand,15.0000,101.0000,2020-01-22,0,0.0,0.0
1,1,NaN,Japan,36.0000,138.0000,2020-01-22,0,0.0,0.0
2,2,NaN,Singapore,1.2833,103.8333,2020-01-22,0,0.0,0.0
3,3,NaN,Nepal,28.1667,84.2500,2020-01-22,0,0.0,0.0
4,4,NaN,Malaysia,2.5000,112.5000,2020-01-22,0,0.0,0.0


In [12]:
#Filter on one location to make sure working correctly
df_melt[df_melt['Province/State'] == 'Beijing'].sort_values('Date')

,id,Province/State,Country/Region,Lat,Long,Date,recovered_to_date,Count_Prev_Day,new_recovered
172,172,Beijing,China,40.1824,116.4142,2020-01-22,0,0.0,0.0
622,172,Beijing,China,40.1824,116.4142,2020-01-23,0,0.0,0.0
1072,172,Beijing,China,40.1824,116.4142,2020-01-24,1,0.0,1.0
1522,172,Beijing,China,40.1824,116.4142,2020-01-25,2,1.0,1.0
1972,172,Beijing,China,40.1824,116.4142,2020-01-26,2,2.0,0.0
2422,172,Beijing,China,40.1824,116.4142,2020-01-27,2,2.0,0.0
2872,172,Beijing,China,40.1824,116.4142,2020-01-28,4,2.0,2.0
3322,172,Beijing,China,40.1824,116.4142,2020-01-29,4,4.0,0.0
3772,172,Beijing,China,40.1824,116.4142,2020-01-30,4,4.0,0.0
4222,172,Beijing,China,40.1824,116.4142,2020-01-31,5,4.0,1.0


In [13]:
#Drop Prev Day Cout column
recovered_df = df_melt.drop(columns=['Count_Prev_Day'])
recovered_df.head()

,id,Province/State,Country/Region,Lat,Long,Date,recovered_to_date,new_recovered
0,0,NaN,Thailand,15.0000,101.0000,2020-01-22,0,0.0
1,1,NaN,Japan,36.0000,138.0000,2020-01-22,0,0.0
2,2,NaN,Singapore,1.2833,103.8333,2020-01-22,0,0.0
3,3,NaN,Nepal,28.1667,84.2500,2020-01-22,0,0.0
4,4,NaN,Malaysia,2.5000,112.5000,2020-01-22,0,0.0


In [14]:
#Reset index and save prvious index as id column
deaths_df.reset_index(inplace=True)
deaths_df.head()

,index,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,...,3/6/20,3/7/20,3/8/20,3/9/20,3/10/20,3/11/20,3/12/20,3/13/20,3/14/20,3/15/20
0,0,NaN,Thailand,15.0000,101.0000,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
1,1,NaN,Japan,36.0000,138.0000,0,0,0,0,0,...,6,6,6,10,10,15,16,19,22,22
2,2,NaN,Singapore,1.2833,103.8333,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,NaN,Nepal,28.1667,84.2500,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,NaN,Malaysia,2.5000,112.5000,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
#Rename to have id column
deaths_df = deaths_df.rename(columns={"index":"id"})
deaths_df.head()

,id,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,...,3/6/20,3/7/20,3/8/20,3/9/20,3/10/20,3/11/20,3/12/20,3/13/20,3/14/20,3/15/20
0,0,NaN,Thailand,15.0000,101.0000,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
1,1,NaN,Japan,36.0000,138.0000,0,0,0,0,0,...,6,6,6,10,10,15,16,19,22,22
2,2,NaN,Singapore,1.2833,103.8333,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,NaN,Nepal,28.1667,84.2500,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,NaN,Malaysia,2.5000,112.5000,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
#Transpose date columns to have as row values
df_melt = deaths_df.melt(id_vars=['id','Province/State','Country/Region','Lat','Long'])
df_melt.columns = ['id','Province/State','Country/Region','Lat','Long','Date','deaths_to_date']
df_melt['Date'] = pd.to_datetime(df_melt['Date'])
df_melt = df_melt.sort_values(['Date','id'])
df_melt['Count_Prev_Day'] = df_melt.sort_values('Date').groupby(['id','Province/State','Country/Region',
                                                                 'Lat','Long'])['deaths_to_date'].shift(1)
df_melt[['Count_Prev_Day']] = df_melt[['Count_Prev_Day']].fillna(value=0)
df_melt['new_deaths'] = df_melt['deaths_to_date'] - df_melt['Count_Prev_Day']

df_melt.head()

,id,Province/State,Country/Region,Lat,Long,Date,deaths_to_date,Count_Prev_Day,new_deaths
0,0,NaN,Thailand,15.0000,101.0000,2020-01-22,0,0.0,0.0
1,1,NaN,Japan,36.0000,138.0000,2020-01-22,0,0.0,0.0
2,2,NaN,Singapore,1.2833,103.8333,2020-01-22,0,0.0,0.0
3,3,NaN,Nepal,28.1667,84.2500,2020-01-22,0,0.0,0.0
4,4,NaN,Malaysia,2.5000,112.5000,2020-01-22,0,0.0,0.0


In [17]:
#Filter on one location to make sure working correctly
df_melt[df_melt['Province/State'] == 'Beijing'].sort_values('Date')

,id,Province/State,Country/Region,Lat,Long,Date,deaths_to_date,Count_Prev_Day,new_deaths
172,172,Beijing,China,40.1824,116.4142,2020-01-22,0,0.0,0.0
622,172,Beijing,China,40.1824,116.4142,2020-01-23,0,0.0,0.0
1072,172,Beijing,China,40.1824,116.4142,2020-01-24,0,0.0,0.0
1522,172,Beijing,China,40.1824,116.4142,2020-01-25,0,0.0,0.0
1972,172,Beijing,China,40.1824,116.4142,2020-01-26,0,0.0,0.0
2422,172,Beijing,China,40.1824,116.4142,2020-01-27,1,0.0,1.0
2872,172,Beijing,China,40.1824,116.4142,2020-01-28,1,1.0,0.0
3322,172,Beijing,China,40.1824,116.4142,2020-01-29,1,1.0,0.0
3772,172,Beijing,China,40.1824,116.4142,2020-01-30,1,1.0,0.0
4222,172,Beijing,China,40.1824,116.4142,2020-01-31,1,1.0,0.0


In [18]:
#Drop prev day count column
deaths_df = df_melt.drop(columns=['Count_Prev_Day'])
deaths_df.head()

,id,Province/State,Country/Region,Lat,Long,Date,deaths_to_date,new_deaths
0,0,NaN,Thailand,15.0000,101.0000,2020-01-22,0,0.0
1,1,NaN,Japan,36.0000,138.0000,2020-01-22,0,0.0
2,2,NaN,Singapore,1.2833,103.8333,2020-01-22,0,0.0
3,3,NaN,Nepal,28.1667,84.2500,2020-01-22,0,0.0
4,4,NaN,Malaysia,2.5000,112.5000,2020-01-22,0,0.0


In [19]:
# Merge confirmed and deaths df on all common columns
semicombined_df = pd.merge(confirmed_df, deaths_df, on=['id','Province/State','Country/Region','Lat','Long', 'Date'])
semicombined_df.head()

,id,Province/State,Country/Region,Lat,Long,Date,confirmed_to_date,new_confirmed,deaths_to_date,new_deaths
0,0,NaN,Thailand,15.0000,101.0000,2020-01-22,2,2.0,0,0.0
1,1,NaN,Japan,36.0000,138.0000,2020-01-22,2,2.0,0,0.0
2,2,NaN,Singapore,1.2833,103.8333,2020-01-22,0,0.0,0,0.0
3,3,NaN,Nepal,28.1667,84.2500,2020-01-22,0,0.0,0,0.0
4,4,NaN,Malaysia,2.5000,112.5000,2020-01-22,0,0.0,0,0.0


In [20]:
#Merge semimerged df w recovered df on all common columns
combined_df = pd.merge(semicombined_df, recovered_df, on=['id','Province/State','Country/Region','Lat','Long','Date'])
combined_df.head()

,id,Province/State,Country/Region,Lat,Long,Date,confirmed_to_date,new_confirmed,deaths_to_date,new_deaths,recovered_to_date,new_recovered
0,0,NaN,Thailand,15.0000,101.0000,2020-01-22,2,2.0,0,0.0,0,0.0
1,1,NaN,Japan,36.0000,138.0000,2020-01-22,2,2.0,0,0.0,0,0.0
2,2,NaN,Singapore,1.2833,103.8333,2020-01-22,0,0.0,0,0.0,0,0.0
3,3,NaN,Nepal,28.1667,84.2500,2020-01-22,0,0.0,0,0.0,0,0.0
4,4,NaN,Malaysia,2.5000,112.5000,2020-01-22,0,0.0,0,0.0,0,0.0


In [21]:
#Rename columns to format for SQL
final_df = combined_df.rename(columns={"id":"id_loc","Province/State": "province_state", "Country/Region": "country_region", "Lat":"lat", "Long": "long", "Date": "date"})
final_df.head()

,id_loc,province_state,country_region,lat,long,date,confirmed_to_date,new_confirmed,deaths_to_date,new_deaths,recovered_to_date,new_recovered
0,0,NaN,Thailand,15.0000,101.0000,2020-01-22,2,2.0,0,0.0,0,0.0
1,1,NaN,Japan,36.0000,138.0000,2020-01-22,2,2.0,0,0.0,0,0.0
2,2,NaN,Singapore,1.2833,103.8333,2020-01-22,0,0.0,0,0.0,0,0.0
3,3,NaN,Nepal,28.1667,84.2500,2020-01-22,0,0.0,0,0.0,0,0.0
4,4,NaN,Malaysia,2.5000,112.5000,2020-01-22,0,0.0,0,0.0,0,0.0


In [24]:
# conection to SPQ database 
rds_connection_string =f"postgres:postgres@localhost:5432/COVID19_DB"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [25]:
engine.table_names()


['covid19']

In [26]:
# table with all data

final_df.to_sql(name='covid19', con=engine, if_exists='replace', index=False)

In [27]:
## Confirm data has been queried properly
pd.read_sql_query('select * from covid19', con=engine).head()

,id,id_loc,province_state,country_region,lat,long,date,confirmed_to_date,new_confirmed,deaths_to_date,new_deaths,recovered_to_date,new_recovered
0,1,0,Anhui,Mainland China,31.8257,117.2264,2020-01-22,1,1,0,0,0,0
1,2,1,Beijing,Mainland China,40.1824,116.4142,2020-01-22,14,14,0,0,0,0
2,3,2,Chongqing,Mainland China,30.0572,107.8740,2020-01-22,6,6,0,0,0,0
3,4,3,Fujian,Mainland China,26.0789,117.9874,2020-01-22,1,1,0,0,0,0
4,5,4,Gansu,Mainland China,36.0611,103.8343,2020-01-22,0,0,0,0,0,0
